In [57]:
import os
from pathlib import Path
import pickle
from dataclasses import dataclass, field
from typing import Any, Dict, List
from datasets import load_dataset
from tqdm import tqdm
import random

In [22]:
with open("/home/alexserra98/helm-suite/inference_id/datasets_overlap_short/mmlu:subject=biology,method=multiple_choice_joint,model=meta-llama_Llama-2-7b-hf,max_train_instances=4/request_states.pkl", "rb") as f:
    request_states = pickle.load(f)

In [46]:
print(request_states[0].request.prompt)


The following are multiple choice questions (with answers) about biology.

Question: According to the pressure-flow model of movement of phloem contents, photosynthate movement from source to sink is driven by
A. an ATP-dependent pressure-flow pump
B. a water-pressure potential gradient
C. transpiration
D. apoplastic diffusion
Answer: B

Question: Homologous structures are often cited as evidence for the process of natural selection. All of the following are examples of homologous structures EXCEPT
A. the wings of a bird and the wings of a bat
B. the flippers of a whale and the arms of a man
C. the pectoral fins of a porpoise and the flippers of a seal
D. the forelegs of an insect and the forelimbs of a dog
Answer: D

Question: A mutation in a bacterial enzyme changed a previously polar amino acid into a nonpolar amino acid. This amino acid was located at a site distant from the enzyme’s active site. How might this mutation alter the enzyme’s substrate specificity?
A. By changing the e

In [28]:
d = "cais/mmlu:all".split(":")
dataset = load_dataset(*d)

Found cached dataset mmlu (/home/alexserra98/.cache/huggingface/datasets/cais___mmlu/all/1.0.0/49beb454d1ef6d7994f43183dc194713f39dd672b4f5a92c08b0b394991910c7)
100%|██████████| 4/4 [00:00<00:00, 1117.88it/s]


In [ ]:
class RequestInstance():
    prompt: str
    letter_gold: str
    token_gold: int = None
    
labels = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5}

In [35]:
dataset["test"][0]

{'question': 'Which of the following best describes the balance the Supreme Court has struck between the establishment clause and the free-exercise clause?',
 'subject': 'high_school_government_and_politics',
 'choices': ['Freedom of speech is protected except in certain situations, such as yelling "fire" in a crowded theater.',
  'Once a church has been recognized by the federal government, its tax-exempt status can never be revoked.',
  'Once Congress has created an administrative agency, that agency can be dissolved only by a constitutional amendment.',
  'State-sponsored prayer during school hours is prohibited, but voluntary prayer by student groups before school is allowed.'],
 'answer': 3}

In [49]:
prompts = []
for row in tqdm(dataset["test"], desc="Constructing Prompts"):
    prompt = f'The following are multiple choice questions (with answers) about {row["subject"]}.\n\n'
    prompt += f'Question: {row["question"]}\n'
    mapping = ['A','B','C','D','E']
    for n, choice in enumerate(row["choices"]):
        prompt += f'{mapping[n]}. {choice}\n'
    prompt += f'Answer: \n\n'
    prompts.append(prompt)
print(prompts[0])

Constructing Prompts: 100%|██████████| 14042/14042 [00:00<00:00, 19868.01it/s]

The following are multiple choice questions (with answers) about high_school_government_and_politics.

Question: Which of the following best describes the balance the Supreme Court has struck between the establishment clause and the free-exercise clause?
A. Freedom of speech is protected except in certain situations, such as yelling "fire" in a crowded theater.
B. Once a church has been recognized by the federal government, its tax-exempt status can never be revoked.
C. Once Congress has created an administrative agency, that agency can be dissolved only by a constitutional amendment.
D. State-sponsored prayer during school hours is prohibited, but voluntary prayer by student groups before school is allowed.
Answer: 




In [60]:
random.seed(42)

In [65]:
@dataclass
class RequestInstance():
    prompt: str
    letter_gold: str
    token_gold: int = None

In [61]:
def construct_question(row,shot=False):
    prompt = f'Question: {row["question"]}\n'
    for n, choice in enumerate(row["choices"]):
        prompt += f'{mapping[n]}. {choice}\n'
    prompt += f'Answer: {mapping[row["answer"]]}\n\n' if shot else  f'Answer:' 
    return prompt 

In [69]:
ri = []
num_instances = 3

mapping = ['A','B','C','D','E']
for row in tqdm(dataset["test"], desc="Constructing Prompts"):
    prompt = f'The following are multiple choice questions (with answers) about {row["subject"]}.\n\n'
    for i in range(num_instances):
        random_row = random.choice(dataset["auxiliary_train"])
        prompt += construct_question(random_row,shot=True)
    prompt += construct_question(row)
    ri.append(RequestInstance(prompt,mapping[row["answer"]]))
    
print(ri[0])

Constructing Prompts: 100%|██████████| 14042/14042 [00:03<00:00, 3933.01it/s]

RequestInstance(prompt='The following are multiple choice questions (with answers) about high_school_government_and_politics.\n\nQuestion: His name was Fleming, and he was a poor Scottish farmer. One day, while trying to eke  out a living for his family, he heard a cry for help coming from a nearby bog  . He dropped his tools and ran to the bog. There, mired to his waist in black muck  was a terrified boy, screaming and struggling to free himself. Farmer Fleming saved the lad from what could have been a slow and terrifying death. The next day, a fancy carriage pulled up to the Scotsman\'s sparse  surroundings. An elegantly dressed nobleman stepped out and introduced himself as the father of the boy Farmer Fleming had saved. "I want to repay you," said the nobleman. "You saved my son\'s life." "No, " the Scottish farmer replied, waving off the offer. At that moment, the farmer\'s own son came to the door of the family hovel . "Is that your son?" the nobleman asked. "Yes," the farmer rep

In [70]:
ri[0].letter_gold

'D'